In [243]:
#python example to infer document vectors from trained doc2vec model

import gensim.models as g
import codecs
import logging
import pandas as pd
import numpy as np
import nltk
import math

from scipy import spatial

import warnings
warnings.filterwarnings('ignore')

In [244]:
import bcolz
import pickle

def load_array(fname):
    return bcolz.open(fname)[:]

def load_vectors(loc):
    return (load_array(loc+'.dat'),
        pickle.load(open(loc+'_words.pkl','rb'),encoding='latin1'),
        pickle.load(open(loc+'_idx.pkl','rb'),encoding='latin1'))



In [2]:

df=pd.read_excel('/home/ubuntu/nbs/intrctbt/el-al-qa/english_qs_with_sim.xlsx')
df=df.fillna('0')

## Training

In [18]:
g.Doc2Vec()

<module 'gensim.models' from '/home/ubuntu/anaconda3/lib/python3.6/site-packages/gensim/models/__init__.py'>

In [20]:

#doc2vec parameters
vector_size = 300
window_size = 15
min_count = 1
sampling_threshold = 1e-5
negative_size = 5
train_epoch = 100
dm = 0 #0 = dbow; 1 = dmpv
worker_count = 1 #number of parallel processes

#pretrained word embeddings
pretrained_emb = "toy_data/pretrained_word_embeddings.txt" #None if use without pretrained embeddings

#input corpus
train_corpus = "toy_data/train_docs.txt"

#output model
saved_path = "toy_data/model.bin"

#enable logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

#train doc2vec model
docs = g.doc2vec.TaggedLineDocument(train_corpus)
model = g.Doc2Vec(docs, size=vector_size, window=window_size, min_count=min_count, sample=sampling_threshold\
                  , workers=worker_count, hs=0, dm=dm, negative=negative_size, dbow_words=1, dm_concat=1\
                  , pretrained_emb=pretrained_emb, iter=train_epoch)

#save model
model.save(saved_path)

TypeError: __init__() got an unexpected keyword argument 'pretrained_emb'

## inference

In [58]:
model="/mnt/doc2vec/enwiki_dbow/doc2vec.bin"
test_docs="/mnt/doc2vec/doc2vec_jhlau_repo/toy_data/test_docs.txt"
output_file="/mnt/doc2vec/doc2vec_jhlau_repo/toy_data/test_vectors.txt"

In [59]:
#parameters

#inference hyper-parameters
start_alpha=0.01
infer_epoch=1000

#load model
m = g.Doc2Vec.load(model)
test_docs = [ x.strip().split() for x in codecs.open(test_docs, "r", "utf-8").readlines() ]

#infer test vectors
output = open(output_file, "w")
for d in test_docs:
    output.write( " ".join([str(x) for x in m.infer_vector(d, alpha=start_alpha, steps=infer_epoch)]) + "\n" )
    print( " ".join([str(x) for x in m.infer_vector(d, alpha=start_alpha, steps=infer_epoch)]) + "\n" )
output.flush()
output.close()

2017-09-20 10:35:04,835 : INFO : loading Doc2Vec object from /mnt/doc2vec/enwiki_dbow/doc2vec.bin
2017-09-20 10:35:07,034 : INFO : loading docvecs recursively from /mnt/doc2vec/enwiki_dbow/doc2vec.bin.docvecs.* with mmap=None
2017-09-20 10:35:07,035 : INFO : loading syn1neg from /mnt/doc2vec/enwiki_dbow/doc2vec.bin.syn1neg.npy with mmap=None
2017-09-20 10:35:07,321 : INFO : loading syn0 from /mnt/doc2vec/enwiki_dbow/doc2vec.bin.syn0.npy with mmap=None
2017-09-20 10:35:07,593 : INFO : setting ignored attribute syn0norm to None
2017-09-20 10:35:07,594 : INFO : setting ignored attribute cum_table to None
2017-09-20 10:35:07,595 : INFO : loaded /mnt/doc2vec/enwiki_dbow/doc2vec.bin


0.0216436 0.0854046 -0.185204 0.0356358 0.0226328 -0.0609381 0.133721 -0.0639421 0.0497548 0.160449 -0.0301217 -0.0432909 -0.114529 -0.0219256 0.0065285 0.0158566 -0.076362 0.00554071 -0.101755 0.300938 0.00578605 0.0564531 0.012077 -0.0799537 0.118507 -0.070728 0.0493632 -0.0236319 0.033678 -0.0267807 -0.180624 -0.092562 -0.155553 -0.0207406 -0.0395616 -0.0960103 0.13248 -0.00721711 0.101279 0.00299204 0.0357573 0.121912 0.051151 -0.138849 0.102214 0.0286217 -0.169847 -0.0858566 -0.0100096 0.0724598 0.0402162 -0.169451 0.0119931 -0.0582123 0.165648 -0.0330144 0.0784302 -0.0629913 -0.0333527 0.184052 0.141084 -0.151428 -0.0601707 -0.0140813 0.0519632 0.0728871 0.0654866 -0.0458783 0.0263909 -0.088683 -0.115823 0.0274354 0.0321476 0.0635585 -0.00865742 0.201109 -0.025348 0.299084 0.0884241 -0.170358 0.0907135 -0.126383 0.0276741 0.0703275 -0.127814 -0.0631541 -0.0897914 -0.107956 0.0133067 0.117487 0.0808917 -0.03296 0.0854023 -0.0596744 -0.0236444 -0.081851 -0.0498654 -0.129927 0.00756

-0.0580145 0.0844129 -0.223962 0.0707562 -0.0581973 -0.0423879 0.310574 0.00274483 0.0380898 0.0514117 0.041907 -0.233334 -0.168289 -0.00599111 0.184991 -0.0154308 0.112005 -0.048468 -0.118082 0.270497 0.123109 0.0103087 0.0839201 0.108659 0.148261 0.081697 -0.0149615 -0.0849192 0.11357 -0.0636976 -0.154875 -0.0215738 -0.198757 0.00264054 0.104954 -0.0752362 0.192261 0.0698167 0.054874 0.0205997 -0.00857907 -0.0497352 0.0810336 -0.182045 0.0074132 -0.0481412 -0.10667 -0.0515832 0.0436978 0.0875907 -0.0961484 -0.301595 0.0955244 0.00116739 0.154345 -0.159124 0.0942137 -0.0663526 -0.0144558 -0.010189 0.156589 -0.125682 -0.0359899 -0.140317 0.0673641 -0.0324498 0.0665124 -0.136701 0.0460653 -0.0321823 -0.0281212 0.0776099 -0.116902 -0.00344283 -0.0261331 0.122416 -0.104572 0.15433 0.0750393 -0.218113 0.141039 -0.0621861 -0.038441 -0.106939 -0.197546 -0.0845887 0.0335277 -0.0105609 -0.0234857 0.124208 -0.0242843 -0.0943142 0.128381 -0.019902 -0.154441 -0.139476 0.118586 -0.115229 0.0409443

0.0183266 0.0628538 -0.187119 -0.0451502 0.0154273 0.0749242 0.0452183 -0.0509705 0.0189996 0.150605 -0.0609305 -0.0113238 -0.103759 0.0728252 -0.0665279 0.0703587 -0.040942 0.0189568 -0.0173925 0.234026 -0.0252618 -0.000921649 0.0672366 -0.042465 0.160138 -0.00210004 -0.0221738 0.0726459 -0.0522018 -0.108438 -0.122558 -0.0192022 -0.0682046 0.0177036 0.03331 -0.0749818 0.155881 -0.00808936 0.0520202 0.0864794 -0.0447216 0.200327 0.02929 -0.0738025 0.123687 0.0153572 -0.15852 -0.044999 -0.00741602 -0.00680136 -0.0947865 -0.0855464 0.0159673 0.0157394 0.176333 0.0319466 0.113519 0.00647322 -0.0738147 0.0899746 0.0380505 0.0518677 -0.0805634 0.0136613 0.0520494 -0.0400053 -0.0672454 -0.046862 0.0343621 -0.0392811 -0.126352 0.0535951 -0.00417347 0.0290788 0.0644625 0.0486651 0.0423755 0.17071 0.123773 -0.0990372 0.0847648 -0.0401259 0.0220568 0.0820116 -0.0797267 -0.0185735 -0.0434638 -0.0838288 -0.0245387 0.0625932 0.0699513 0.0753737 0.0596684 -0.0267959 -0.000584549 -0.182111 -0.045617 

In [84]:
test_docs[0]

['the',
 'cardigan',
 'welsh',
 'corgi',
 'is',
 'one',
 'of',
 'two',
 'separate',
 'dog',
 'breeds',
 'known',
 'as',
 'welsh',
 'corgis',
 'that',
 'originated',
 'in',
 'wales',
 ',',
 'the',
 'other',
 'being',
 'the',
 'pembroke',
 'welsh',
 'corgi',
 '.',
 'it',
 'is',
 'one',
 'of',
 'the',
 'oldest',
 'herding',
 'breeds',
 '.']

In [3]:
model_path='/mnt/doc2vec/wiki_word2vec_skipgram/word2vec.bin'
word2vec_model=g.Word2Vec.load(model_path)

In [85]:
m.most_similar(test_docs[0])

[('and', 0.9134107828140259),
 ('a', 0.8979176878929138),
 ('which', 0.8863018751144409),
 ('from', 0.8839833736419678),
 ('also', 0.8786206245422363),
 ('to', 0.8762887120246887),
 ('this', 0.8658530712127686),
 ('with', 0.8636301755905151),
 ('an', 0.8599680662155151),
 ('well', 0.8533967733383179)]

In [80]:
word2vec_model.most_similar('eilat')

[('ashdod', 0.7111448645591736),
 ('ashkelon', 0.6947978734970093),
 ('haifa', 0.6847111582756042),
 ('tel', 0.6671017408370972),
 ('aviv', 0.6644912362098694),
 ('kiryat', 0.6576217412948608),
 ('israeli', 0.6565908193588257),
 ('nahariya', 0.6549043655395508),
 ('negev', 0.6450300216674805),
 ('israel', 0.6363789439201355)]

In [56]:
len(m.vocab)

669549

In [54]:
m.similar_by_vector(a,d)

2017-09-20 10:33:27,841 : INFO : precomputing L2-norms of word weight vectors


KeyError: "word 'Is there a flight to Amsterdam?' not in vocabulary"

In [50]:
m.infer_vector(a)

array([  1.60444085e-03,  -4.33947280e-04,  -6.86381536e-04,
        -1.00000342e-03,  -1.41199876e-03,  -1.19081722e-03,
         2.67490948e-04,   1.38619682e-04,  -4.54126915e-04,
         9.84983868e-04,   1.19066320e-03,   4.24988684e-05,
        -6.76351192e-04,  -1.22101884e-03,   7.64075725e-04,
        -2.56646599e-04,  -7.15418544e-04,   4.25992155e-04,
        -4.18097305e-04,  -4.74409142e-04,  -7.92906678e-04,
         1.07538537e-04,   1.05508414e-04,   1.34872145e-03,
         1.01886014e-03,  -2.38334076e-04,  -1.50690204e-03,
        -1.46528659e-03,  -1.38606341e-03,  -5.04577823e-04,
         3.03850713e-04,  -6.45410328e-04,   1.98570109e-04,
         1.11528463e-03,  -1.28776301e-03,  -4.62782395e-04,
         5.74739242e-04,   5.92469121e-04,  -9.32098948e-04,
         3.00850981e-04,   1.29858078e-03,  -1.83380704e-04,
        -8.99472041e-04,  -1.34076574e-03,  -1.24931091e-03,
        -1.36683753e-03,   1.53012795e-03,  -1.54409005e-04,
         1.14815065e-03,

In [49]:
a='Is there a flight to Amsterdam?','Is there a flight to Amsterdam?'

In [5]:
df.Question.head()

0    I have a continuation flight on the same day. ...
1    How can I reserve seats on continuation flights? 
2            Do I earn points on continuation flights?
3                   How do I select or change my seat?
4              When can I submit a bid for an upgrade?
Name: Question, dtype: object

In [6]:
qs={}
for q in list(df.Question):
    qs[q]=m.infer_vector(q)

NameError: name 'm' is not defined

In [64]:
qs.keys()

dict_keys(['I have a continuation flight on the same day. Can I check in online?', 'How can I reserve seats on continuation flights? ', 'Do I earn points on continuation flights?', 'How do I select or change my seat?', 'When can I submit a bid for an upgrade?', 'When can check-in be performed?', 'For UP flights, is it possible to arrive at Terminal 3 instead of Terminal1?', 'What is the difference between Economy Class and Reduced-Price Economy Class?', "Why can't I order baggage service on the UP website?", 'I sent a query and no one got back to me. Why? ', 'What is Passenger Profile?', 'Is it possible to issue an invoice for a flight ticket?', 'What benefits do I receive for upgrading to Business Class?', 'How can I see where I will be seated on the plane?', 'Can I go back to earlier stages during the reservation procedure?', 'How can I correct a name after marriage?', 'How can I correct the spelling of a name?', 'Can I correct the spelling of a name on the reservation?', 'Is it perm

## w2v stuff

In [87]:
most_sim_sentence('where is my flight',list(df.Question))

['I have a continuation flight on the same day. Can I check in online?',
 'How can I reserve seats on continuation flights? ',
 'Do I earn points on continuation flights?',
 'How do I select or change my seat?',
 'When can I submit a bid for an upgrade?',
 'When can check-in be performed?',
 'For UP flights, is it possible to arrive at Terminal 3 instead of Terminal1?',
 'What is the difference between Economy Class and Reduced-Price Economy Class?',
 "Why can't I order baggage service on the UP website?",
 'I sent a query and no one got back to me. Why? ',
 'What is Passenger Profile?',
 'Is it possible to issue an invoice for a flight ticket?',
 'What benefits do I receive for upgrading to Business Class?',
 'How can I see where I will be seated on the plane?',
 'Can I go back to earlier stages during the reservation procedure?',
 'How can I correct a name after marriage?',
 'How can I correct the spelling of a name?',
 'Can I correct the spelling of a name on the reservation?',
 'Is

In [93]:
words=list(df.Question)[0].split()

In [97]:
len(word2vec_model[words[0].lower()])

300

In [15]:
vec_list=[]
for sent in list(df.Question):
    sent_vec=np.zeros(300)
    for word in nltk.word_tokenize(sent):
        try:
            sent_vec+=word2vec_model[word.lower()]
        except:
            print(word.lower(),' not in vocab')
    vec_list.append(sent_vec)

#TODO: add matmid to vocab, split better

terminal1  not in vocab
matmid  not in vocab
matmid  not in vocab
matmid  not in vocab
matmid  not in vocab
class/first  not in vocab
purchase/s  not in vocab
matmid  not in vocab
matmid  not in vocab
matmid  not in vocab
matmid  not in vocab
matmid  not in vocab
matmid  not in vocab
matmid  not in vocab
baby/infant  not in vocab
al-up  not in vocab
matmid  not in vocab
matmid  not in vocab
(  not in vocab
)  not in vocab
al-up  not in vocab
order/cancel  not in vocab
order/cancel  not in vocab
cancel/change  not in vocab
change/cancel  not in vocab
dynamicnewsletterforsidewidget  not in vocab
matmid  not in vocab
(  not in vocab
)  not in vocab
matmid  not in vocab
matmid  not in vocab
matmid  not in vocab
cancel/change  not in vocab
change/cancel  not in vocab
dreamstream  not in vocab
dreamstream  not in vocab
dreamstream  not in vocab
dreamstream  not in vocab
dreamstream  not in vocab
dreamstream  not in vocab
dreamstream  not in vocab
dreamstream  not in vocab
dreamstream  not in

In [11]:
qs=list(df.Question)

In [117]:
from scipy import spatial

for i,vec_1 in enumerate(vec_list):
    for j,vec_2 in enumerate(vec_list[i:]):
        pass
        #print(1 - spatial.distance.cosine(vec_1, vec_2),list(df.Question)[i],'X',list(df.Question)[i+j])

In [9]:
#find most similar sentences


In [31]:
for j in range(10):
    rand=np.random.randint(416)
    print('anchor: ',qs[rand])
    lis=[]
    for i,vec in enumerate(vec_list):
        if i!=rand:
            sim=1 - spatial.distance.cosine(vec_list[rand], vec)
            if math.isnan(sim):
                sim=0
            lis.append(sim)
        else:
            lis.append(0)
    
    max_index=np.argmax(lis)
    print('closest match: ',qs[max_index])
    print('')

anchor:  What is Business Class Seating Service?
closest match:  What is First Class Seating Service?

anchor:  How is a bonus ticket reserved for Matmid Club members?
closest match:  How long is a bonus ticket valid?

anchor:  Can I purchase a Preferred Seat in Economy Class with Matmid Club points?
closest match:  What is a Preferred Seat in Economy Class?

anchor:  What is an Extra Comfort Seat for which I am submitting a bid?
closest match:  When can I submit a bid for an Extra Seat?

anchor:  Can I bring one suitcase for two people?
closest match:  I sent a query and no one got back to me. Why? 

anchor:  Can a suitcase be routed to its final destination?
closest match:  How can I fly to one destination and return from another one?

anchor:  I ordered a Preferred Seat; where should I go when I arrive at the Terminal?
closest match:  What should I do if I change my reservation (flight, date) and I have already ordered a Preferred Seat?

anchor:  What are the conditions of my ticket

In [52]:
word2vec_model['A']

KeyError: "word 'A' not in vocabulary"

In [ ]:
model_path='/mnt/doc2vec/wiki_word2vec_skipgram/word2vec.bin'
model=g.Word2Vec.load(model_path)

## Sentence_matcher class

In [ ]:
class model_glove:
    __init__(self):
        self.item
    
    __getitem__(self,key):
        

In [252]:
glove_model={}
for word in words:
    glove_model[word]= vecs[wordidx[word]]

In [ ]:
from sen

In [1]:
from sentence_matcher import Sentence_matcher

ModuleNotFoundError: No module named 'sentence_matcher'

In [4]:
import bcolz
import pickle

def load_array(fname):
    return bcolz.open(fname)[:]

def load_vectors(loc):
    return (load_array(loc+'.dat'),
        pickle.load(open(loc+'_words.pkl','rb'),encoding='latin1'),
        pickle.load(open(loc+'_idx.pkl','rb'),encoding='latin1'))

In [1]:
import gensim.models as g
import codecs
import logging
import pandas as pd
import numpy as np
import nltk
import math
import re
from scipy import spatial

import warnings
warnings.filterwarnings('ignore')

class Sentence_matcher:
    
    def __init__(self,question_list,model='word2vec'): #models=word2vec,glove
        
        self.vec_list=[]
        self.question_list=question_list
            
        if model=='word2vec':
        #load the average word2vec model
            model_path='/mnt/doc2vec/wiki_word2vec_skipgram/word2vec.bin'
            self.model=g.Word2Vec.load(model_path)
        
        if model=='glove':
            vecs, words, wordidx = load_vectors('/mnt/glove/6B.300d')
            self.model={}
            for word in words:
                self.model[word]= vecs[wordidx[word]]
            
        for sent in question_list:
            sent_vec=np.zeros(300)
            for word in self.ad_hok_parse(sent):
                try:
                    sent_vec+=self.model[word.lower()]

                except:
                    print(word.lower(),' not in vocab')
            self.vec_list.append(sent_vec)   
        #TODO: add not in vocab words
        
    
    def get_closest_match(self,query,n=1):
        #embed query
        sent_vec=self.sent2vec(query)

        #find closest match
        lis=[]
        for i,vec in enumerate(self.vec_list):

                sim=1 - spatial.distance.cosine(sent_vec, vec)
                if math.isnan(sim):
                    sim=0
                lis.append(sim)


        max_index=np.argmax(lis)
        print('closest match: ',self.question_list[max_index])
        return self.question_list[max_index] 
    
    def sent2vec(self,sentence):
        #use word2vec sum curenlty
        sent_vec=np.zeros(300)
        for word in self.ad_hok_parse(sentence):
            try:
                sent_vec+=self.model[word.lower()]
            except:
                print(word.lower(),' not in vocab')
        return sent_vec
    
    def ad_hok_parse(self,sentence):
        #parsing sentences
        print(sentence)
        output=[]
        words=nltk.word_tokenize(sentence)
        for word in words:

            if word.lower()=='matmid':
                word='frequent'
            words=re.split('[1/-]',word)

            output+=words

        return output
    
    def similarity(self,word1,word2):
        sim=1 - spatial.distance.cosine(self.model[word1], self.model[word2])
        return sim

In [5]:
sm_glove=Sentence_matcher(list(df.Question),model='glove')


I have a continuation flight on the same day. Can I check in online?
How can I reserve seats on continuation flights? 
Do I earn points on continuation flights?
How do I select or change my seat?
When can I submit a bid for an upgrade?
When can check-in be performed?
For UP flights, is it possible to arrive at Terminal 3 instead of Terminal1?
  not in vocab
What is the difference between Economy Class and Reduced-Price Economy Class?
Why can't I order baggage service on the UP website?
I sent a query and no one got back to me. Why? 
What is Passenger Profile?
Is it possible to issue an invoice for a flight ticket?
What benefits do I receive for upgrading to Business Class?
How can I see where I will be seated on the plane?
Can I go back to earlier stages during the reservation procedure?
How can I correct a name after marriage?
How can I correct the spelling of a name?
Can I correct the spelling of a name on the reservation?
Is it permitted to change a name in the reservation?
How can 

In [6]:
sm_word2vec=Sentence_matcher(list(df.Question),model='word2vec')

I have a continuation flight on the same day. Can I check in online?
How can I reserve seats on continuation flights? 
Do I earn points on continuation flights?
How do I select or change my seat?
When can I submit a bid for an upgrade?
When can check-in be performed?
For UP flights, is it possible to arrive at Terminal 3 instead of Terminal1?
  not in vocab
What is the difference between Economy Class and Reduced-Price Economy Class?
Why can't I order baggage service on the UP website?
I sent a query and no one got back to me. Why? 
What is Passenger Profile?
Is it possible to issue an invoice for a flight ticket?
What benefits do I receive for upgrading to Business Class?
How can I see where I will be seated on the plane?
Can I go back to earlier stages during the reservation procedure?
How can I correct a name after marriage?
How can I correct the spelling of a name?
Can I correct the spelling of a name on the reservation?
Is it permitted to change a name in the reservation?
How can 

In [207]:
sm.get_closest_match('I want a better seat')

I want a better seat
closest match:  Will I be able to choose my seat and my Extra Seat?


'Will I be able to choose my seat and my Extra Seat?'

In [41]:
def get_closest_match(query):
    
    #embed query
    sent_vec=sent2vec(query)

    #find closest match
    lis=[]
    for i,vec in enumerate(vec_list):
        if i!=rand:
            sim=1 - spatial.distance.cosine(sent_vec, vec)
            if math.isnan(sim):
                sim=0
            lis.append(sim)
        else:
            lis.append(0)
    
    max_index=np.argmax(lis)
    print('closest match: ',qs[max_index])
    return qs[max_index]

In [44]:
def sent2vec(sentence):
    #use word2vec sum curenlty
    sent_vec=np.zeros(300)
    for word in nltk.word_tokenize(sentence):
        try:
            sent_vec+=word2vec_model[word.lower()]
        except:
            print(word.lower(),' not in vocab')
    return sent_vec

In [43]:
sent2vec('how can I find my seat')

## similarity matrix

In [31]:
np.array(sm.vec_list).shape

(415, 300)

In [84]:
df.iloc[34].Question

'What is the FLY CARD credit card and how can I sign up for one?'

In [69]:
# find most similar sentences
import itertools

sim_list=[1 - spatial.distance.cosine(i, j) for i,j in itertools.product(sm.vec_list,sm.vec_list)]

In [68]:
sm.vec_list[220][3]=0.5 # hack to prevent nullity. TODO:permanent sollution

In [72]:
sim_array=np.array(sim_list)

In [74]:
sim_array[sim_array>0.999]=0 

In [63]:
sim_array

array([ 0.        ,  0.83074504,  0.8070891 , ...,  0.56006044,
        0.59294969,  0.        ])

In [76]:
sim_array[14145]

0.99036450310633728

In [75]:
np.argmax(sim_array)

14145

In [89]:
top_sim_indixes=np.argsort(sim_array)[::-1][:5]

In [90]:
for i in top_sim_indixes:
    i_1=i % 415
    i_2=i //415
    print(df.iloc[i_1].Question)
    print(df.iloc[i_2].Question)
    print(' ')

What is the FLY CARD PREMIUM credit card and how can I sign up for one?
What is the FLY CARD credit card and how can I sign up for one?
 
What is the FLY CARD credit card and how can I sign up for one?
What is the FLY CARD PREMIUM credit card and how can I sign up for one?
 
How can I submit a bid for an Extra Seat?
When can I submit a bid for an Extra Seat?
 
When can I submit a bid for an Extra Seat?
How can I submit a bid for an Extra Seat?
 
If I submit bids for both flight segments, when will I receive notification that my bid for an Extra Seat has or hasn't been approved?
If I submitted bids for both flight segments, when will I receive notification that my bid for an upgrade has or hasn't been approved?
 


In [78]:
mat=np.array(sim_array).reshape(415,415)

In [93]:
np.argsort(mat[0])[::-1][:5]

array([ 48, 335,  47,  45, 326])

In [55]:
np.argsort(mat)[::-1]

array([[414,  77, 161, ..., 233, 156, 220],
       [413, 329,  29, ..., 111, 147, 220],
       [412,  29, 197, ...,  27,  28, 220],
       ..., 
       [  2, 207, 311, ..., 221, 324, 220],
       [  1, 207,  92, ..., 305, 175, 220],
       [  0, 207,  92, ..., 335,  48, 220]])

In [94]:
df.iloc[0].Question

'I have a continuation flight on the same day. Can I check in online?'

In [96]:
df.iloc[48].Question

'I made an order on the Duty Free website, but then had to change the date of my flight. How do I change the flight details on my order?'

In [97]:
df.iloc[335].Question

'Can I book a flight for money on the Matmid website?'

In [100]:
model['xyzz']=1

TypeError: 'Word2Vec' object does not support item assignment

## Labels

In [185]:
tags=[item for sublist in list(df.Labels.str.split(',')) for item in sublist]

In [188]:
Counter(tags)

Counter({' Baggage': 5,
         ' Booking Many Tickets or Combined Ticket': 1,
         ' Canceling Flight Ticket': 1,
         ' Contact': 7,
         ' EL AL Upgrade': 2,
         ' Express Check-in': 5,
         ' Extra Seat': 4,
         ' FAQ-FB': 9,
         ' FAQ-SW': 3,
         ' FLY CARD': 4,
         ' Flights with Continuation Destinations': 3,
         ' Matmid Club': 9,
         ' Ordering Preferred Seat': 4,
         ' Passenger Information': 1,
         ' Passengers with Special Needs': 4,
         ' Reservation Changes': 5,
         ' Reservation Process': 5,
         ' Seat Selection': 2,
         ' Tourism Services': 2,
         '0': 2,
         'Baggage': 6,
         'Booking Many Tickets or Combined Ticket': 2,
         'Canceling Flight Ticket': 6,
         'Contact': 24,
         'Duty Free': 15,
         'EL AL Destinations': 61,
         'EL AL Upgrade': 22,
         'Express Check-in': 14,
         'Extra Seat': 17,
         'FLY CARD': 2,
         'Flights w

In [182]:
list(itertools.chain(list(df.Labels.str.split(','))))

[['Express Check-in', ' Flights with Continuation Destinations'],
 ['Flights with Continuation Destinations', ' Seat Selection'],
 ['Matmid Club', ' Flights with Continuation Destinations'],
 ['Linked', ' FAQ-FB', ' Seat Selection'],
 ['Contact', ' EL AL Upgrade'],
 ['Express Check-in', ' FAQ-FB'],
 ['UP'],
 ['Reservation Process'],
 ['UP'],
 ['Contact', ' FAQ-FB'],
 ['Matmid Club'],
 ['Reservation Process'],
 ['EL AL Upgrade'],
 ['Passenger Information'],
 ['Reservation Process'],
 ['Contact', ' Reservation Changes'],
 ['Contact', ' Reservation Changes'],
 ['Contact', ' Reservation Changes'],
 ['Contact', ' Reservation Changes'],
 ['Reservation Changes'],
 ['Linked', ' Passengers with Special Needs'],
 ['Matmid Club'],
 ['EL AL Upgrade'],
 ['Matmid Club'],
 ['Ordering Preferred Seat'],
 ['Ordering Preferred Seat'],
 ['Ordering Preferred Seat'],
 ['Ordering Preferred Seat'],
 ['Ordering Preferred Seat'],
 ['Contact', ' FLY CARD'],
 ['Matmid Club'],
 ['Contact', ' FLY CARD'],
 ['FLY CAR

## Test similarities

In [294]:
result_list=[]
for i in df.iterrows():
    if i[1].Sim_1!='0' and i[1].Sim_1!=0:
        match=sm_glove.get_closest_match(i[1].Sim_1)
        result_list.append(match==i[1].Question)
    
        #print('right match is:',i[1].Question)
        print(match==i[1].Question)

can I check online for connection flight?
closest match:  Who can check in online for the flight?
False
I want to save seats for a connection
closest match:  Do I have to pay for submitting a bid for an Extra Seat?
False
how does matmid word for connections?
closest match:  How can I use Matmid points for another person?
False
I want to switch seat
closest match:  Will I be able to choose my seat and my Extra Seat?
False
How can I upgrade
closest match:  How can I know whether I can bid for an upgrade?
False
I want to check in
closest match:  Do I have to bring my luggage for check-in by myself?
False
how can I fly from terminal 3 with up?
closest match:  How do I get from Terminal 3 to Terminal 1?
False
the baggage service for up doesnt work
closest match:  Why can't I order baggage service on the UP website?
True
I did not get an answer for my query
closest match:  I sent a query and no one got back to me. Why? 
True
can you provide an invoice for flight ticket?
closest match:  Is it

In [295]:
result_list

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 False,
 True,
 False,
 True,
 True,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 False]

In [292]:
result_list

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 True,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False]

In [283]:
sm_word2vec.similarity('connection','continuation')

0.34158291018545262

In [284]:
sm_glove.similarity('paris','jerusalem')

0.27721219382837681